# Final Proj

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mrsteamedbun/AIPI590/blob/main/Final/Final.ipynb)

In [9]:
!pip install magenta pretty_midi shap lime numpy pandas matplotlib

  Using cached librosa-0.7.2-py3-none-any.whl
  Using cached mir_eval-0.7-py3-none-any.whl
  Using cached numba-0.49.1-cp39-cp39-win_amd64.whl
  Using cached pretty_midi-0.2.9-py3-none-any.whl
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [1 lines of output]
  ERROR: Can not execute `setup.py` since setuptools is not available in the build environment.
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

Encountered error while generating package metadata.

See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [5]:
import pretty_midi
import numpy as np

def extract_features(midi_file):
    midi_data = pretty_midi.PrettyMIDI(midi_file)
    # Extract tempo
    tempo = np.mean(midi_data.get_tempo_changes()[1])  # Average tempo
    # Extract harmony (chord progression)
    chords = []
    for instrument in midi_data.instruments:
        if not instrument.is_drum:
            chords.extend(instrument.notes)
    harmony = [(note.pitch, note.start, note.end) for note in chords]
    return tempo, harmony

# Example usage
midi_file = 'midi/Arabesque-in-E-Nr-1.mid'
tempo, harmony = extract_features(midi_file)
print(f"Tempo: {tempo}, Harmony: {harmony[:10]}")


Tempo: 120.0, Harmony: [(64, 2.216145833333333, 2.5260416666666665), (69, 2.3828125, 2.5625), (61, 2.0, 2.5625), (73, 2.536458333333333, 2.708333333333333), (76, 2.708333333333333, 2.833333333333333), (78, 2.825520833333333, 2.984375), (75, 3.1197916666666665, 3.3151041666666665), (80, 2.9765625, 3.333333333333333), (71, 3.270833333333333, 3.3619791666666665), (68, 3.3854166666666665, 3.5260416666666665)]


In [ ]:
from magenta.models.performance_rnn import performance_sequence_generator
from magenta.protobuf import generator_pb2
from magenta.models.shared import sequence_generator_bundle
import note_seq

def load_performance_rnn(bundle_path):
    """Load a pre-trained PerformanceRNN model."""
    bundle = sequence_generator_bundle.read_bundle_file(bundle_path)
    generator = performance_sequence_generator.PerformanceRnnSequenceGenerator(
        model_name='performance_with_dynamics',
        bundle=bundle
    )
    generator.initialize()
    return generator

# Load the model
model_path = 'performance_rnn.mag'
performance_rnn = load_performance_rnn(model_path)

In [8]:
import shap
import numpy as np

# Dummy function for a model's prediction (replace with your model)
def model_predict(features):
    tempo, harmony = features
    return [tempo * 0.5 + len(harmony) * 0.1]  # Example logic

# SHAP explanation
features = [tempo, harmony]
explainer = shap.KernelExplainer(model_predict, np.array(features).reshape(1, -1))
shap_values = explainer.shap_values(features)
shap.summary_plot(shap_values, features, feature_names=['Tempo', 'Harmony'])

Provided model function fails when applied to the provided data set.


C:\Users\Administrator\AppData\Local\Temp\ipykernel_37680\1643825005.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  explainer = shap.KernelExplainer(model_predict, np.array(features).reshape(1, -1))


ValueError: not enough values to unpack (expected 2, got 1)

In [7]:
from lime import lime_tabular

# Create a dummy dataset (replace with your actual data)
import pandas as pd
data = pd.DataFrame({
    'Tempo': [tempo],
    'Harmony': [len(harmony)]  # Example: Use the length of harmony
})

# LIME explanation
explainer = lime_tabular.LimeTabularExplainer(data.values, feature_names=data.columns)
explanation = explainer.explain_instance(data.iloc[0].values, model_predict)
explanation.show_in_notebook()

NameError: name 'model_predict' is not defined